##  Data Preprocessing

### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display # Allows the use of display() for DataFrames
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelBinarizer

# Show all of the data in a dataframe
pd.set_option('display.max_columns', None)

In [2]:
# Training Set
application_train = pd.read_csv('data/application_train.csv')
print("Loaded Training Set: {0} rows".format(application_train.shape[0]))

application_test = pd.read_csv('data/application_test.csv')
print("Loaded Training Set: {0} rows".format(application_test.shape[0]))

columns = pd.read_csv('data/HomeCredit_columns_description.csv')

Loaded Training Set: 307511 rows
Loaded Training Set: 48744 rows


In [3]:
# Load the list of features indentified in the exploration step which need preprocessing
non_numeric_features = pd.read_csv('data/tmp/non_numeric_features.csv', header=0, index_col=0, names=["feature"])
numeric_features = pd.read_csv('data/tmp/numeric_features.csv', header=0, index_col=0, names=["feature"])
string_to_bool_features = pd.read_csv('data/tmp/string_to_bool.csv', header=0, index_col=0, names=["feature"])
log_transform_features = pd.read_csv('data/tmp/log_transform.csv', header=0, index_col=0, names=["feature"])

In [4]:
display(numeric_features.head())

,feature
0,REGION_POPULATION_RELATIVE
1,DAYS_BIRTH
2,DAYS_EMPLOYED
3,DAYS_REGISTRATION
4,DAYS_ID_PUBLISH


In [5]:
'''
Scales a list of numeric features to a range of [0 .. 1] without changing the distribution of the data.  
Accepts a List of column names.  Returns a new dataFrame.
''' 
def scale_features(dataFrame, featureList): 
    dataFrame_transform = pd.DataFrame(data = dataFrame)
    dataFrame_transform[featureList] = scaler.fit_transform(dataFrame_transform[featureList])
    return dataFrame_transform

In [6]:
''' 
Applies an in-place transformation that converts a Y/N field to binary 1/0
'''
def make_bool(dataFrame, featureName): 
    
    lb = LabelBinarizer()
    bool_name = "BOOL_{0}".format(featureName)    
    dataFrame[bool_name] = lb.fit_transform(dataFrame[featureName])
    result = dataFrame.drop(featureName, axis=1) 
    
    return dataFrame

In [7]:
''' 
Applies an in-place transformation that converts NaN, Inf and -Inf to numeric values
'''
def make_numeric(dataFrame, featureName): 
        
    dataFrame[featureName] = np.nan_to_num(dataFrame[featureName])
    
    return dataFrame

In [8]:
'''
Applies an in-place log transformation to numeric features
'''
def log_transform(dataFrame, featureName):     
    
    transformed = dataFrame[featureName].apply(lambda x: np.log(x + 1))
    
    transformed_name = "LOG_{0}".format(featureName)

    print("Transformed Name: {0}".format(transformed_name))
    
    dataFrame[transformed_name] = transformed
    result = dataFrame.drop(featureName, axis=1)
    
    return dataFrame

## Preprocessing

### Individual Problematic Features

In [9]:
# DAYS_EMPLOYED

# A bunch of records indicate that the person has worked for ~100 years.
# I'm just going to zero them out, because they're invalid
# NOTE: NaN might ultimately be a better choice

application_train['DAYS_EMPLOYED'] = application_train['DAYS_EMPLOYED'].replace({365243: 0})
application_test['DAYS_EMPLOYED'] = application_test['DAYS_EMPLOYED'].replace({365243: 0})

### Convert Y/N String Fields to Boolean

In [10]:
# Convert any Y/N string fields to boolean
for feature in string_to_bool_features['feature']: 
    print("Making Boolean: {0}".format(feature))
    application_test = make_bool(application_test, feature)
    application_train = make_bool(application_train, feature)

Making Boolean: FLAG_OWN_CAR
Making Boolean: FLAG_OWN_REALTY
Making Boolean: FLAG_EMP_PHONE
Making Boolean: FLAG_WORK_PHONE
Making Boolean: FLAG_PHONE
Making Boolean: FLAG_EMAIL


### One-Hot Encode Non-Numeric Features

In [11]:
nnf = non_numeric_features['feature']
print(nnf)

# One-Hot Encode all of our non-numeric features
application_test = pd.get_dummies(application_test, columns=nnf)
application_train = pd.get_dummies(application_train, columns=nnf)

print("Training Set Columns: {0}".format(application_train.shape[1]))
print("Testing Set Columns: {0}".format(application_test.shape[1]))

# Ensure that train and test sets have the same number of columns
# from https://stackoverflow.com/questions/41335718/keep-same-dummy-variable-in-training-and-testing-data/41339045
application_train,application_test = application_train.align(application_test, join='outer', axis=1, fill_value=0)

print("Aligned Training Set Columns: {0}".format(application_train.shape[1]))
print("Aligned Testing Set Columns: {0}".format(application_test.shape[1]))

0                   CODE_GENDER
1               FLAG_OWN_REALTY
2               NAME_TYPE_SUITE
3              NAME_INCOME_TYPE
4           NAME_EDUCATION_TYPE
5            NAME_FAMILY_STATUS
6             NAME_HOUSING_TYPE
7               OCCUPATION_TYPE
8    WEEKDAY_APPR_PROCESS_START
9             ORGANIZATION_TYPE
Name: feature, dtype: object
Training Set Columns: 238
Testing Set Columns: 234
Aligned Training Set Columns: 238
Aligned Testing Set Columns: 238


### Log Transform Skewed Numeric Features

In [12]:
# Convert any Y/N string fields to boolean
for feature in log_transform_features['feature']: 
        print("Performing Log Transformations on: {0}".format(feature))
        application_test = log_transform(application_test, feature)
        application_train = log_transform(application_train, feature)


Performing Log Transformations on: CNT_CHILDREN
Transformed Name: LOG_CNT_CHILDREN
Transformed Name: LOG_CNT_CHILDREN
Performing Log Transformations on: AMT_INCOME_TOTAL
Transformed Name: LOG_AMT_INCOME_TOTAL
Transformed Name: LOG_AMT_INCOME_TOTAL
Performing Log Transformations on: AMT_CREDIT
Transformed Name: LOG_AMT_CREDIT
Transformed Name: LOG_AMT_CREDIT
Performing Log Transformations on: AMT_ANNUITY
Transformed Name: LOG_AMT_ANNUITY
Transformed Name: LOG_AMT_ANNUITY
Performing Log Transformations on: AMT_GOODS_PRICE
Transformed Name: LOG_AMT_GOODS_PRICE
Transformed Name: LOG_AMT_GOODS_PRICE
Performing Log Transformations on: OWN_CAR_AGE
Transformed Name: LOG_OWN_CAR_AGE
Transformed Name: LOG_OWN_CAR_AGE
Performing Log Transformations on: OBS_30_CNT_SOCIAL_CIRCLE
Transformed Name: LOG_OBS_30_CNT_SOCIAL_CIRCLE
Transformed Name: LOG_OBS_30_CNT_SOCIAL_CIRCLE
Performing Log Transformations on: DEF_30_CNT_SOCIAL_CIRCLE
Transformed Name: LOG_DEF_30_CNT_SOCIAL_CIRCLE
Transformed Name: LOG_

### Scale Numeric Fields

In [19]:
numeric = [] # We need a list of all numeric features
ltf = log_transform_features['feature']
nf = numeric_features['feature']

# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler() # default=(0, 1)

for feature in ltf:
    numeric.append("LOG_{0}".format(feature))

for feature in nf: 
    numeric.append("{0}".format(feature))
    
print(numeric)
display(application_train.head())

# Scaling only works if the values are all numeric (no NaNs, infs, etc)
for feature in numeric: 
    application_train = make_numeric(application_train, feature)
    application_test = make_numeric(application_test, feature)
    
application_train[numeric] = scaler.fit_transform(application_train[numeric])
application_test[numeric] = scaler.fit_transform(application_test[numeric])

['LOG_CNT_CHILDREN', 'LOG_AMT_INCOME_TOTAL', 'LOG_AMT_CREDIT', 'LOG_AMT_ANNUITY', 'LOG_AMT_GOODS_PRICE', 'LOG_OWN_CAR_AGE', 'LOG_OBS_30_CNT_SOCIAL_CIRCLE', 'LOG_DEF_30_CNT_SOCIAL_CIRCLE', 'LOG_OBS_60_CNT_SOCIAL_CIRCLE', 'LOG_DEF_60_CNT_SOCIAL_CIRCLE', 'LOG_DAYS_LAST_PHONE_CHANGE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH']


,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_MODE,BOOL_FLAG_EMAIL,BOOL_FLAG_EMP_PHONE,BOOL_FLAG_OWN_CAR,BOOL_FLAG_OWN_REALTY,BOOL_FLAG_PHONE,BOOL_FLAG_WORK_PHONE,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,ELEVATORS_AVG,ELEVATORS_MEDI,ELEVATORS_MODE,EMERGENCYSTATE_MODE,ENTRANCES_AVG,ENTRANCES_MEDI,ENTRANCES_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_2,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_EMAIL,FLAG_EMP_PHONE,FLAG_MOBIL,FLAG_OWN_CAR,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,FLAG_PHONE,FLAG_WORK_PHONE,FLOORSMAX_AVG,FLOORSMAX_MEDI,FLOORSMAX_MODE,FLOORSMIN_AVG,FLOORSMIN_MEDI,FLOORSMIN_MODE,FONDKAPREMONT_MODE,HOUR_APPR_PROCESS_START,HOUSETYPE_MODE,LANDAREA_AVG,LANDAREA_MEDI,LANDAREA_MODE,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,LIVINGAPARTMENTS_AVG,LIVINGAPARTMENTS_MEDI,LIVINGAPARTMENTS_MODE,LIVINGAREA_AVG,LIVINGAREA_MEDI,LIVINGAREA_MODE,NAME_CONTRACT_TYPE,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_AVG,NONLIVINGAREA_MEDI,NONLIVINGAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Culture,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Hotel,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Industry: type 1,ORGANIZATION_TYPE_Industry: type 10,ORGANIZATION_TYPE_Industry: type 11,ORGANIZATION_TYPE_Industry: type 12,ORGANIZATION_TYPE_Industry: type 13,ORG

### Preview
Show our transformed dataset

In [20]:
# Show an example of a record with scaling applied
display(application_train.head(n = 5))

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_MODE,BOOL_FLAG_EMAIL,BOOL_FLAG_EMP_PHONE,BOOL_FLAG_OWN_CAR,BOOL_FLAG_OWN_REALTY,BOOL_FLAG_PHONE,BOOL_FLAG_WORK_PHONE,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,ELEVATORS_AVG,ELEVATORS_MEDI,ELEVATORS_MODE,EMERGENCYSTATE_MODE,ENTRANCES_AVG,ENTRANCES_MEDI,ENTRANCES_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_2,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_EMAIL,FLAG_EMP_PHONE,FLAG_MOBIL,FLAG_OWN_CAR,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,FLAG_PHONE,FLAG_WORK_PHONE,FLOORSMAX_AVG,FLOORSMAX_MEDI,FLOORSMAX_MODE,FLOORSMIN_AVG,FLOORSMIN_MEDI,FLOORSMIN_MODE,FONDKAPREMONT_MODE,HOUR_APPR_PROCESS_START,HOUSETYPE_MODE,LANDAREA_AVG,LANDAREA_MEDI,LANDAREA_MODE,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,LIVINGAPARTMENTS_AVG,LIVINGAPARTMENTS_MEDI,LIVINGAPARTMENTS_MODE,LIVINGAREA_AVG,LIVINGAREA_MEDI,LIVINGAREA_MODE,NAME_CONTRACT_TYPE,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_AVG,NONLIVINGAREA_MEDI,NONLIVINGAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Culture,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Hotel,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Industry: type 1,ORGANIZATION_TYPE_Industry: type 10,ORGANIZATION_TYPE_Industry: type 11,ORGANIZATION_TYPE_Industry: type 12,ORGANIZATION_TYPE_Industry: type 13,ORG

### Output
Save out our preprocessed data to temporary intermediate files

In [21]:
application_train.to_csv('data/tmp/application_train_preprocessed.csv')
application_test.to_csv('data/tmp/application_test_preprocessed.csv')